In [1]:
import re
import joblib
import jieba
import pandas as pd
from sklearn import tree
import jieba.posseg as psg
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# 原始数据
# 尝试使用 utf-8 编码读取数据集
# dataset = pd.read_csv('weibo_senti_100k.csv', encoding='utf-8')
# 如果 utf-8 失败，尝试使用 'gb18030' 编码（是 'gbk' 的扩展版本）
dataset = pd.read_csv('weibo_senti_100k.csv', encoding='gb18030')


In [3]:
dataset

,label,review
0,1,?更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你]
1,1,土耳其的事要认真对待[哈哈]，否则直接开除 很是细心，酒店都全部OK啦。
2,1,姑娘都羡慕你呢…还有招财猫高兴……[哈哈]小学徒一枚，等着明天见您呢，大佬范儿[书呆子]
3,1,美~~~~~[爱你]
4,1,梦想有多大，舞台就有多大![鼓掌]
...,...,...
119983,0,一公里不到，县医院那个天桥下右拐200米就到了！//@谢礼恒: 我靠。这个太霸道了！离224...
119984,0,今天真冷啊，难道又要穿棉袄了[晕]？今年的春天真的是百变莫测啊[抓狂]
119985,0,最近几天就没停止过！！！[伤心]
119986,0,//@毒药女流氓:[怒] 很惨!


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
X = ['?更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你]',
     '土耳其的事要认真对待[哈哈]，否则直接开除 很是细心，酒店都全部OK啦。',
     '姑娘都羡慕你呢…还有招财猫高兴……[哈哈]小学徒一枚，等着明天见您呢，大佬范儿[书呆子]']
# 使用TF-IDF进行特征转换
vect = TfidfVectorizer()
x_train = vect.fit_transform(X)
# 将结果转换为稠密矩阵，并输出
dtm = x_train.todense()
dtm

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.26726124, 0.26726124, 0.        ,
         0.26726124, 0.26726124, 0.80178373, 0.26726124, 0.        ,
         0.        , 0.        ],
        [0.        , 0.46735098, 0.35543247, 0.46735098, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.46735098,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.46735098],
        [0.38988801, 0.        , 0.29651988, 0.        , 0.38988801,
         0.38988801, 0.38988801, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.38988801,
         0.38988801, 0.        ]])

In [5]:
    # 使用jieba进行分词，保留词语和词性
series_recutword = dataset['review'].apply(lambda s: [(x.word, x.flag) for x in psg.cut(s)])
# 统计各评论词数并存储
series_cutwordnum = series_recutword.apply(len)
indexes = list(series_recutword.index)
# 构造分词索引
list_cutwordidx = [idx + 1 for idx, count in zip(indexes, series_cutwordnum) for _ in range(count)]
# 分词及词性拉平
list_cutwordatt = [item for sublist in series_recutword for item in sublist]
list_cutword = [x[0] for x in list_cutwordatt]
list_wordatt = [x[1] for x in list_cutwordatt]
# 标签拉长
list_cutwordlabel = [label for label, count in zip(dataset['label'], series_cutwordnum) for _ in range(count)]
# 构造分词表
data_frame_recutwordtable = pd.DataFrame({
    '分词索引': list_cutwordidx,
    '词语': list_cutword,
    '词性': list_wordatt,
    '标签': list_cutwordlabel
}).query("词性 != 'x'")
# 导入停用词
with open('./stoplist.txt', 'r', encoding='UTF-8') as stop_path:
    stop_words = set(stop_path.read().splitlines())
# 得到不含停用词的分词表
data_frame_recutwordtable = data_frame_recutwordtable[data_frame_recutwordtable['词语'].isin(stop_words)]
data_frame_recutwordtable

IndentationError: expected an indented block after 'with' statement on line 22 (2571852570.py, line 23)

In [6]:
import pandas as pd
import re
from jieba import posseg as psg

# 读取停用词
with open('./stoplist.txt', 'r', encoding='UTF-8') as stop_path:
    stop_words = set(stop_path.read().split())

# 初始化一个空的DataFrame来存储处理后的数据
processed_data = pd.DataFrame()

for index, row in dataset.iterrows():
    # 清洗文本，只保留中文字符
    cleaned_text = re.sub(r'[^\u4e00-\u9fa5]', '', row['review'])
    # 使用jieba进行分词，保留词语和词性
    words_pos = [(x.word, x.flag) for x in psg.cut(cleaned_text) if x.word not in stop_words and x.word.strip()]
    # 创建一个临时列表来存储当前review的处理结果
    temp_data_list = []
    for word, flag in words_pos:
        temp_data_list.append({
            '分词索引': index + 1,
            '词语': word,
            '词性': flag,
            '标签': row['label']
        })
    # 将当前review的处理结果转换为DataFrame
    temp_data = pd.DataFrame(temp_data_list)
    # 将当前review的DataFrame添加到总的DataFrame中
    processed_data = pd.concat([processed_data, temp_data], ignore_index=True)

processed_data


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Angle\AppData\Local\Temp\jieba.cache
Loading model cost 0.710 seconds.
Prefix dict has been built successfully.


,分词索引,词语,词性,标签
0,1,更博,v,1
1,1,爆照,v,1
2,1,帅,a,1
3,1,越来越,d,1
4,1,爱,v,1
...,...,...,...,...
1791559,119988,乜,nr,0
1791560,119988,鬼,n,0
1791561,119988,知入,v,0
1791562,119988,睇,vg,0


In [7]:
X = processed_data.groupby('分词索引')['词语'].apply(list).tolist()
# 提取每条评论的标签列表，取第一个标签作为单个样本的标签
Y = processed_data.groupby('分词索引')['标签'].first().tolist()

In [8]:
X

[['更博', '爆照', '帅', '越来越', '爱', '生', '傻', '缺爱', '爱', '爱'],
 ['土耳其', '认真对待', '开除', '细心'],
 ['姑娘', '羡慕', '招财猫', '学徒', '一枚', '明天', '大佬', '范儿', '书呆子'],
 ['美', '爱'],
 ['梦想', '舞台', '鼓掌'],
 ['花心', '鼓掌', '春暖花开'],
 ['问答',
  '社区',
  '收到',
  '大学生',
  '发给',
  '私信',
  '偶',
  '喜欢',
  '阿姨',
  '偶是',
  '阿姨',
  '控',
  '回',
  '阿姨',
  '稀饭',
  '盆友',
  '偶是',
  '盆友',
  '控'],
 ['吃货',
  '无不',
  '啧啧称奇',
  '喜欢',
  '写错',
  '字',
  '森林',
  '小天使',
  '波琪',
  '厦门',
  '摄影师',
  '日月星辰',
  '心',
  '路上',
  '每种',
  '型号',
  '生',
  '两胎',
  '志远',
  '天下',
  '行',
  '监控',
  '防盗',
  '安装',
  '创意',
  '美食',
  '哥',
  '漫游者',
  '强子',
  '陈',
  '小猫',
  '游子',
  '歌',
  '厦门'],
 ['爱', '享受', '生活', '微笑', '嘻嘻哈哈', '挤眼', '开心', '早安', '甜心'],
 ['恭喜', '祝福', '小江江', '爱里', '健康', '成长', '爱'],
 ['鼓掌',
  '慕春彦',
  '一流',
  '经纪',
  '公司',
  '超模',
  '摇篮',
  '鼓掌',
  '姚戈',
  '东方',
  '宾利',
  '强大',
  '名模',
  '军团'],
 ['真', '可爱', '害羞'],
 ['第一次',
  '见到',
  '花瓣',
  '面膜',
  '抵',
  '面膜',
  '好几片',
  '补水',
  '神器',
  '一帖',
  '见效',
  '睡',
  '分钟',
  '超',
  '神奇',
  '膜法',
  '第二天',


In [9]:
Y

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [17]:
# 训练集、测试集划分
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

# 将分词列表转为一维字符串列表，每个元素代表一个评论的分词拼接结果
x_train = [' '.join(words) for words in x_train]
x_test = [' '.join(words) for words in x_test]

# 词转向量，one-hot编码
count_vec = CountVectorizer(binary=False, decode_error="replace",max_df=0.5)

# 训练集One-hot编码
x_train_one_hot = count_vec.fit_transform(x_train)

# 测试集One-hot编码
x_test_one_hot = count_vec.transform(x_test)

# 查看训练集One-hot编码结果的前几行
print(x_train_one_hot[:5, :])

# 或者将One-hot编码结果转换为稀疏矩阵的密集表示，以便更直观地查看
x_train_dense = x_train_one_hot.toarray()
print(x_train_dense[:5])

  (0, 41)	1
  (0, 123282)	1
  (0, 126172)	1
  (0, 90774)	1
  (0, 61430)	1
  (0, 126420)	1
  (0, 28786)	1
  (0, 108151)	1
  (0, 107485)	1
  (0, 39592)	1
  (0, 119777)	1
  (0, 32039)	1
  (1, 92843)	1
  (1, 3153)	1
  (1, 107758)	1
  (1, 107759)	1
  (1, 86078)	1
  (1, 28879)	1
  (1, 51312)	1
  (1, 89762)	1
  (1, 118963)	1
  (1, 46078)	1
  (1, 93285)	1
  (1, 91232)	1
  (1, 104059)	1
  :	:
  (3, 1875)	1
  (3, 72982)	1
  (3, 15912)	1
  (3, 14043)	1
  (3, 37712)	1
  (3, 10121)	1
  (3, 7537)	1
  (3, 98516)	1
  (3, 82286)	1
  (3, 59511)	1
  (3, 41470)	1
  (3, 27955)	1
  (3, 78681)	1
  (3, 53178)	1
  (3, 124629)	1
  (3, 40277)	1
  (3, 8360)	1
  (4, 32039)	1
  (4, 5384)	1
  (4, 36537)	2
  (4, 5127)	1
  (4, 36521)	1
  (4, 58815)	1
  (4, 19441)	1
  (4, 135830)	1


MemoryError: Unable to allocate 102. GiB for an array with shape (95840, 142244) and data type int64

In [14]:
#构建决策树
dtc=tree.DecisionTreeClassifier(max_depth=10)
dtc.fit(x_train,y_train)
y_true=y_test
y_pred=dtc.predict(x_test)
print('在测试集上的准确率：%.2f'% accuracy_score(y_true,y_pred))

在测试集上的准确率：0.69


In [15]:
for depth in range(1,250):
    dtc=tree.DecisionTreeClassifier(max_depth=depth)
    dtc.fit(x_train,y_train)
    y_true=y_test
    y_pred=dtc.predict(x_test)
    print('深度为'+ str(depth) +'时准确率：%.5f'% accuracy_score(y_true,y_pred))

深度为1时准确率：0.67367
深度为2时准确率：0.68014
深度为3时准确率：0.68010
深度为4时准确率：0.68357
深度为5时准确率：0.68370
深度为6时准确率：0.68357
深度为7时准确率：0.68348
深度为8时准确率：0.68474
深度为9时准确率：0.68617
深度为10时准确率：0.68587
深度为11时准确率：0.68622
深度为12时准确率：0.68535
深度为13时准确率：0.68639
深度为14时准确率：0.68639
深度为15时准确率：0.68600
深度为16时准确率：0.68513
深度为17时准确率：0.68504
深度为18时准确率：0.68461
深度为19时准确率：0.68517
深度为20时准确率：0.68561
深度为21时准确率：0.68444
深度为22时准确率：0.68465
深度为23时准确率：0.68465
深度为24时准确率：0.68509
深度为25时准确率：0.68474
深度为26时准确率：0.68535
深度为27时准确率：0.68444
深度为28时准确率：0.68539
深度为29时准确率：0.68557
深度为30时准确率：0.68544
深度为31时准确率：0.68448
深度为32时准确率：0.68461
深度为33时准确率：0.68379
深度为34时准确率：0.68257
深度为35时准确率：0.68240
深度为36时准确率：0.68140
深度为37时准确率：0.68127
深度为38时准确率：0.67966
深度为39时准确率：0.68092
深度为40时准确率：0.67970
深度为41时准确率：0.67923
深度为42时准确率：0.67866
深度为43时准确率：0.67732
深度为44时准确率：0.67762
深度为45时准确率：0.67806
深度为46时准确率：0.67784
深度为47时准确率：0.67788
深度为48时准确率：0.67753
深度为49时准确率：0.67732
深度为50时准确率：0.67740
深度为51时准确率：0.67641
深度为52时准确率：0.67714
深度为53时准确率：0.67636
深度为54时准确率：0.67628
深度为55时准确率：0.67619
深度为56时准确率：0.67584
深

KeyboardInterrupt: 

In [15]:
for depth in range(250,350):
    dtc=tree.DecisionTreeClassifier(max_depth=depth)
    dtc.fit(x_train,y_train)
    y_true=y_test
    y_pred=dtc.predict(x_test)
    print('深度为'+ str(depth) +'时准确率：%.5f'% accuracy_score(y_true,y_pred))

在训练集上的准确率：0.69
              precision    recall  f1-score   support

           0       0.63      0.91      0.74       210
           1       0.81      0.40      0.54       190

    accuracy                           0.67       400
   macro avg       0.72      0.66      0.64       400
weighted avg       0.71      0.67      0.64       400

在测试集上的准确率：0.67


在训练集上的准确率：0.69
              precision    recall  f1-score   support

           0       0.63      0.91      0.74       210
           1       0.81      0.40      0.54       190

    accuracy                           0.67       400
   macro avg       0.72      0.66      0.64       400
weighted avg       0.71      0.67      0.64       400

在测试集上的准确率：0.67


在训练集上的准确率：0.71
              precision    recall  f1-score   support

           0       0.63      0.90      0.74       210
           1       0.78      0.41      0.54       190

    accuracy                           0.67       400
   macro avg       0.70      0.65      0.64       400
weighted avg       0.70      0.67      0.64       400

在测试集上的准确率：0.67


在训练集上的准确率：0.73
              precision    recall  f1-score   support

           0       0.65      0.87      0.74       210
           1       0.76      0.47      0.58       190

    accuracy                           0.68       400
   macro avg       0.70      0.67      0.66       400
weighted avg       0.70      0.68      0.67       400

在测试集上的准确率：0.68


在训练集上的准确率：0.74
              precision    recall  f1-score   support

           0       0.63      0.89      0.74       210
           1       0.78      0.43      0.56       190

    accuracy                           0.67       400
   macro avg       0.71      0.66      0.65       400
weighted avg       0.70      0.67      0.65       400

在测试集上的准确率：0.67


在测试集上的准确率：0.67


在测试集上的准确率：0.69


在测试集上的准确率：0.69


在测试集上的准确率：0.67
